In [1]:
# !pip install deeppavlov emoji
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install tensorflow==2.15.1
# !pip install transformers
# !export PATH=/usr/local/cuda-12.4/bin:$PATH
# !export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH

In [2]:
!pip install transformers pandas torch spacy yake datasets

In [3]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=1alondqI-2IHo__mYU7KQz4Ip8ytYGHXg'
output_file_name = 'wildberries_reviews.csv'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv' уже существует.


In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Загрузка необходимых ресурсов nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# !pip uninstall tensorflow tensorflow-gpu -y
# !pip uninstall torch torchvision torchaudio -y
# # Установка TensorFlow
# !pip install tensorflow==2.12.0

# # Установка PyTorch с поддержкой CUDA 12.1
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-keyring_1.1-1_all.deb
# !dpkg -i cuda-keyring_1.1-1_all.deb
# !apt-get update
# !apt-get -y install cudnn
# !export PATH=/usr/local/cuda/bin:$PATH
# !export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH

In [6]:
import tensorflow as tf
import torch

print(f"TensorFlow version: {tf.__version__}")
print(f"Available GPUs for TensorFlow: {tf.config.list_physical_devices('GPU')}")

print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available for PyTorch: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version for PyTorch: {torch.version.cuda}")
    print(f"cuDNN version for PyTorch: {torch.backends.cudnn.version()}")


2024-08-03 05:09:56.578616: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-03 05:09:56.727646: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-03 05:09:56.729032: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-03 05:09:57.707072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0
Available GPUs for TensorFlow: []
PyTorch version: 2.4.0+cu121
Is CUDA available for PyTorch: True
CUDA version for PyTorch: 12.1
cuDNN version for PyTorch: 90100


2024-08-03 05:10:01.792582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-03 05:10:01.793363: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [25]:
import pandas as pd
import re
import yake
from tqdm import tqdm
import nltk
from nltk.tokenize import sent_tokenize
from spacy.lang.ru.stop_words import STOP_WORDS as stopwords_ru
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from IPython.display import display
import spacy
import time
import emoji
import os

nltk.download('punkt')
print(f"torch.cuda.is_available() - {torch.cuda.is_available()}")

nlp = spacy.load("ru_core_news_lg")

def clean_text(text):
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'<.*?>', '', text)  # Удаление HTML-тегов
    text = re.sub(r'\s+', ' ', text).strip()  # Удаление лишних пробелов и обрезка
    return text

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Удаление знаков пунктуации
    text = text.lower()  # Приведение текста к нижнему регистру
    tokens = text.split()  # Токенизация
    tokens = [word for word in tokens if word not in stopwords_ru]  # Удаление стоп-слов
    return ' '.join(tokens)

kw_extractor = yake.KeywordExtractor(lan="ru", n=1, dedupLim=0.9, top=10)

def extract_aspects(text):
    keywords = kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords]

print("Загрузка модели и токенизатора...")
model_name = "cointegrated/rubert-tiny-sentiment-balanced"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
print("Модель и токенизатор загружены.")

narichiya_stepeni = [
    "очень", "сильно", "чрезвычайно", "весьма", "крайне", "исключительно", "необычайно", "колоссально", 
    "неимоверно", "ужасно", "безмерно", "невероятно", "совершенно", "абсолютно", "капитально", "дико", 
    "неописуемо", "бесконечно", "потрясающе", "несказанно", "сказочно", "фантастически", "гигантски", 
    "необыкновенно", "супер", "чрезмерно", "изумительно", "отлично", "восхитительно", "превосходно", 
    "грандиозно", "максимально", "поразительно", "бесподобно", "безгранично", "огромно", "гиперболически", 
    "значительно", "существенно", "немыслимо", "экстремально"
]

sravnitelno_usilitelnye_konstruktsii = [
    "настолько", "так", "столь", "чуть-чуть", "немного", "вполне", "очень", "более", "менее", "гораздо", 
    "намного", "чуть", "слишком", "сверх", "сильно", "крайне", "безмерно", "необычайно", "невероятно", 
    "чересчур", "едва", "еле", "весьма", "особенно", "предельно", "максимально", "почти"
]

def get_lemmas(doc):
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(lemmas)

def map_review_rating(rating):
    if rating in [4, 5]:
        return 'positive'
    else:
        return 'negative'

def analyze_aspects(doc, rating_sentiment):
    text = doc.text
    lemmatized_text = get_lemmas(doc)
    aspects = extract_aspects(lemmatized_text)
    aspect_details = {}
    for aspect in aspects:
        aspect_lemma = aspect
        sentences = [sent for sent in doc.sents if aspect in sent.text]
        if sentences:
            sentiment_result = classifier(aspect)
            aspect_sentiment = sentiment_result[0]['label']
            combined_sentiment = aspect_sentiment if rating_sentiment == 'neutral' else rating_sentiment
            aspect_desc = max(sentences, key=lambda sent: len([token for token in sent if token.text == aspect])).text
            characteristics = []

            def collect_related_words(token):
                related_words = []
                for child in token.children:
                    if child.dep_ in {"amod", "acomp", "advmod", "prep"} or child.pos_ in {"ADJ", "ADV", "VERB"}:
                        if child.dep_ == "neg":
                            related_words.append(f"не_{child.head.lemma_}")
                        elif child.text.lower() in narichiya_stepeni or child.text.lower() in sravnitelno_usilitelnye_konstruktsii:
                            related_words.append(f"{child.text.lower()}_{child.head.lemma_}")
                        else:
                            related_words.append(child.lemma_)
                        related_words.extend(collect_related_words(child))
                return related_words

            for token in sentences[0]:
                if token.head.text == aspect or token.text == aspect:
                    characteristics.extend(collect_related_words(token))
            
            characteristics = list(dict.fromkeys(characteristics))  # Удаление дубликатов, сохраняя порядок

            description = f"{aspect_desc}. Тональность: {combined_sentiment}. Характеристики: {', '.join(characteristics)}."
            if aspect_lemma not in aspect_details:
                aspect_details[aspect_lemma] = {
                    'sentiment': combined_sentiment,
                    'description': description,
                    'characteristics': characteristics,
                    'count': 1,
                    'rating_positive': 1 if rating_sentiment == 'positive' else 0,
                    'rating_negative': 1 if rating_sentiment == 'negative' else 0
                }
            else:
                aspect_details[aspect_lemma]['count'] += 1
                aspect_details[aspect_lemma]['description'] += f" | {description}"
                aspect_details[aspect_lemma]['characteristics'].extend(characteristics)
                if rating_sentiment == 'positive':
                    aspect_details[aspect_lemma]['rating_positive'] += 1
                elif rating_sentiment == 'negative':
                    aspect_details[aspect_lemma]['rating_negative'] += 1
    return aspect_details

def process_texts(texts, ratings, batch_size):
    all_aspect_details = []
    for doc, rating in tqdm(zip(nlp.pipe(texts, batch_size=batch_size), ratings), total=len(texts), desc="Processing docs"):
        rating_sentiment = map_review_rating(rating)
        all_aspect_details.append(analyze_aspects(doc, rating_sentiment))
    return all_aspect_details

def evaluate_aspect_importance(df):
    aspect_counter = {}
    for _, row in df.iterrows():
        product = row['product']
        if product not in aspect_counter:
            aspect_counter[product] = {}
        for aspect, details in row['aspect_details'].items():
            if aspect not in aspect_counter[product]:
                aspect_counter[product][aspect] = {
                    'count': 0,
                    'positive': 0,
                    'negative': 0,
                    'neutral': 0,
                    'descriptions': [],
                    'characteristics': [],
                    'rating_positive': 0,
                    'rating_negative': 0,
                    'sentiment': details['sentiment']
                }
            aspect_counter[product][aspect]['count'] += details['count']
            aspect_counter[product][aspect][details['sentiment']] += 1
            aspect_counter[product][aspect]['descriptions'].append(details['description'])
            aspect_counter[product][aspect]['characteristics'].extend(details['characteristics'])
            aspect_counter[product][aspect]['rating_positive'] += details['rating_positive']
            aspect_counter[product][aspect]['rating_negative'] += details['rating_negative']
    return aspect_counter

def analyze_category(category, df):
    # Фильтрация данных по категории
    category_df = df[df['category'] == category].copy()
    if category_df.empty:
        print(f"No data found for category: {category}")
        return
    
    # Преобразование текстов и анализ аспектов
    print(f"Analyzing category: {category}")
    category_df['processed_text'] = category_df['corrected_text'].apply(preprocess_text)
    category_df['sentences'] = category_df['corrected_text'].apply(sent_tokenize)
    category_df['rating_sentiment'] = category_df['review_rating'].apply(map_review_rating)
    
    texts = category_df["corrected_text"].apply(clean_text).tolist()
    ratings = category_df["review_rating"].tolist()
    all_aspect_details = process_texts(texts, ratings, batch_size=512)
    
    category_df['aspect_details'] = all_aspect_details
    
    # Оценка важности аспектов
    aspect_importance = evaluate_aspect_importance(category_df)
    
    # Создание итоговой таблицы для каждого товара
    product_aspects = []
    for product, aspects in aspect_importance.items():
        for aspect, metrics in aspects.items():
            characteristics_counter = {char: metrics['characteristics'].count(char) for char in set(metrics['characteristics'])}
            if not characteristics_counter:
                continue
            sorted_characteristics = sorted(characteristics_counter.items(), key=lambda x: x[1], reverse=True)
            sorted_characteristics_str = ', '.join([f"{char}: {count}" for char, count in sorted_characteristics])
            total_characteristics_count = sum(characteristics_counter.values())
            product_aspects.append({
                'product': product,
                'aspect': aspect,
                'count': metrics['count'],
                'positive': metrics['positive'],
                'negative': metrics['negative'],
                'neutral': metrics['neutral'],
                'descriptions': max(metrics['descriptions'], key=len),
                'characteristics': sorted_characteristics_str,
                'total_characteristics_count': total_characteristics_count,
                'rating_positive': metrics['rating_positive'],
                'rating_negative': metrics['rating_negative'],
                'sentiment': metrics['sentiment']
            })

    aspect_df = pd.DataFrame(product_aspects)
    aspect_df = aspect_df.sort_values(by=['product', 'total_characteristics_count'], ascending=[True, False])
    
    # Сводка по категории
    category_summary = aspect_df.groupby('product').agg({
        'count': 'sum',
        'positive': 'sum',
        'negative': 'sum',
        'neutral': 'sum',
        'rating_positive': 'sum',
        'rating_negative': 'sum'
    }).reset_index()
    
    def get_top_aspects(df, sentiment, top_n=5):
        filtered_df = df[df['sentiment'] == sentiment]
        top_aspects = filtered_df.sort_values(by='count', ascending=False).head(top_n)['aspect'].tolist()
        return ', '.join(top_aspects)
    
    category_summary['positive_aspects'] = category_summary['product'].apply(lambda x: get_top_aspects(aspect_df[aspect_df['product'] == x], 'positive'))
    category_summary['negative_aspects'] = category_summary['product'].apply(lambda x: get_top_aspects(aspect_df[aspect_df['product'] == x], 'negative'))
    
    category_summary['positive_ratio'] = category_summary['positive'] / category_summary['count']
    category_summary['negative_ratio'] = category_summary['negative'] / category_summary['count']
    category_summary = category_summary.sort_values(by='positive_ratio', ascending=False)

    # Выбор популярных и менее популярных товаров
    popular_threshold = max(5, int(len(category_summary) * 0.25))
    popular_products = category_summary.nlargest(popular_threshold, 'count')
    less_popular_products = category_summary[(category_summary['count'] >= 10) & (~category_summary.index.isin(popular_products.index))]

    # Рекомендация товаров
    popular_recommendation = popular_products.iloc[0]
    less_popular_recommendation = less_popular_products.iloc[0]

    recommendations = {
        'popular': {
            'product': popular_recommendation['product'],
            'positive_ratio': popular_recommendation['positive_ratio'],
            'rating_positive': popular_recommendation['rating_positive'],
            'rating_negative': popular_recommendation['rating_negative'],
            'positive_aspects': popular_recommendation['positive_aspects'],
            'negative_aspects': popular_recommendation['negative_aspects']
        },
        'less_popular': {
            'product': less_popular_recommendation['product'],
            'positive_ratio': less_popular_recommendation['positive_ratio'],
            'rating_positive': less_popular_recommendation['rating_positive'],
            'rating_negative': less_popular_recommendation['rating_negative'],
            'positive_aspects': less_popular_recommendation['positive_aspects'],
            'negative_aspects': less_popular_recommendation['negative_aspects']
        }
    }

    print(f"\nИтоговая таблица важных аспектов для каждого товара в категории '{category}':")
    display(aspect_df.head(10))
    
    print(f"\nСводка по категории '{category}':")
    display(category_summary.head(10))
    
    print(f"\nРекомендованный популярный товар в категории '{category}':")
    display(recommendations['popular'])

    print(f"\nРекомендованный менее популярный товар в категории '{category}':")
    display(recommendations['less_popular'])
    
    return aspect_df, category_summary, recommendations

print("Загрузка данных...")
start_time = time.time()
df = pd.read_csv('wildberries_reviews.csv', nrows=1000)
print("Данные загружены. Время выполнения: {:.2f} секунд".format(time.time() - start_time))

# Анализ категории
category = df.iloc[0].category
aspect_df, category_summary, recommendations = analyze_category(category, df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


torch.cuda.is_available() - True
Загрузка модели и токенизатора...
Модель и токенизатор загружены.
Загрузка данных...
Данные загружены. Время выполнения: 0.01 секунд
Analyzing category: /Автотовары/OFFroad


Processing docs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703/703 [00:07<00:00, 98.95it/s]



Итоговая таблица важных аспектов для каждого товара в категории '/Автотовары/OFFroad':


,product,aspect,count,positive,negative,neutral,descriptions,characteristics,total_characteristics_count,rating_positive,rating_negative,sentiment
151,Autobrand_AED / Дополнительная led фара 30w с ...,моргать,1,0,1,0,"Но через месяц сгорел один диод, второй ещё че...",постоянно: 1,1,0,1,negative
164,Hangkai / Лебедка электрическая влагозащитная ...,заменить,1,1,0,0,"Провод плюсовой тонкий 10мм, лучше сразу замен...","сразу: 1, иначе: 1, больший: 1, расплавиться: 1",4,1,0,positive
174,Hangkai / Лебедка электрическая влагозащитная ...,лебёдка,3,3,0,0,"Очень хорошая лебёдка, справляется отлично.. Т...","хороший: 2, отличный: 1, очень_хороший: 1",4,3,0,positive
175,Hangkai / Лебедка электрическая влагозащитная ...,доставка,3,3,0,0,Отзыв после установки и испытания Лебёдка приш...,"установить: 1, заказывать: 1, и: 1, испытать: 1",4,3,0,positive
157,Hangkai / Лебедка электрическая влагозащитная ...,спорт,1,1,0,0,В Казахстан Костанай скую область город Лисако...,"установить: 1, исчерпать: 1, проверить: 1",3,1,0,positive
158,Hangkai / Лебедка электрическая влагозащитная ...,трос,4,2,2,0,"Здравствуйте, товар пришел в целости и сохранн...","стальной: 1, вот: 1, измерять: 1",3,2,2,positive
163,Hangkai / Лебедка электрическая влагозащитная ...,опрессован,1,1,0,0,Пришлось заменить один наконечник кабельный т....,"не: 1, нормально: 1, кабельный: 1",3,1,0,positive
166,Hangkai / Лебедка электрическая влагозащитная ...,кольцо,1,1,0,0,"Рычаг свободного хода лебедки не герметичный, ...","резиновый: 1, уплотнительный: 1, больший: 1",3,1,0,positive
168,Hangkai / Лебедка электрическая влагозащитная ...,упаковка,5,5,0,0,"Лебедку получили, упаковка достойная, все герм...","отличный: 1, очень_хороший: 1, хороший: 1",3,5,0,positive
162,Hangkai / Лебедка электрическая влагозащитная ...,крутоймужу,1,1,0,0,Очень крутоймужу понравилсявся комплектация вс...,"очень_крутоймужу: 1, понравилсявся: 1",2,1,0,positive


...

,product,count,positive,negative,neutral,rating_positive,rating_negative,positive_aspects,negative_aspects,positive_ratio,negative_ratio
2,MOTORin / Расширители арок 60 Эк,5,5,0,0,5,0,"хватить, резинка, товар, кусочек, ворота",,1.000000,0.000000
3,MOTORin / Расширитель колёсных арок 40 мм,7,7,0,0,7,0,"метр, контур, прилегать, соответствие, дубит",,1.000000,0.000000
6,Shtapler / Лебедка электрическая 12v 3500lb 15...,14,14,0,0,14,0,"лебёдка, товар, оку, заказывать, подъём",,1.000000,0.000000
7,Shtapler / Лебедка электрическая 12v 4500lb 20...,14,14,0,0,14,0,"лебёдка, товар, оку, заказывать, подъём",,1.000000,0.000000
8,Vixem / Лебедка автомобильная 12 v 4000 1814 к...,1,1,0,0,1,0,кадрик,,1.000000,0.000000
4,MOTORin / расширители арок,12,11,1,0,11,1,"комплект, вид, грязь, полный, штука",,0.916667,0.083333
10,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,107,93,14,0,93,14,"вид, вещь, пластик, качество, товар","чайка, застрять, взгляд, грязь, привод",0.869159,0.130841
9,ВПМ / Антибукс - антипробуксовочные траки утол...,120,103,17,0,103,17,"вещь, комплект, вид, выехать, колесо","фото, морочьте, результат, создайте, проблема",0.858333,0.141667
11,ПК ЛИМ / Браслеты цепи противоскольжения,53,43,10,0,43,10,"комплект, замок, товар, выехать, цена","обидно, переплатить, неудобный, крючка, аккуратно",0.811321,0.188679
12,ПК ЛИМ / Цепи противоскольжения для легковых а...,53,43,10,0,43,10,"комплект, замок, товар, выехать, цена","обидно, переплатить, неудобный, крючка, аккуратно",0.811321,0.188679



Рекомендованный популярный товар в категории '/Автотовары/OFFroad':


{'product': 'ВПМ / Антибукс - антипробуксовочные траки утолщенные (2 шт)',
 'positive_ratio': 0.8583333333333333,
 'rating_positive': 103,
 'rating_negative': 17,
 'positive_aspects': 'вещь, комплект, вид, выехать, колесо',
 'negative_aspects': 'фото, морочьте, результат, создайте, проблема'}


Рекомендованный менее популярный товар в категории '/Автотовары/OFFroad':


{'product': 'Shtapler / Лебедка электрическая 12v 3500lb 1588кг 15м',
 'positive_ratio': 1.0,
 'rating_positive': 14,
 'rating_negative': 0,
 'positive_aspects': 'лебёдка, товар, оку, заказывать, подъём',
 'negative_aspects': ''}